In [ ]:
import os
import json
import pandas as pd
import traceback


In [ ]:
conda install -p c:\Users\imava\MCQgenerator\env ipykernel --update-deps --force-reinstall

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from dotenv import load_dotenv

load_dotenv  #take environment variables from .env


In [ ]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

# Till now we have made the setup and initialised our llm , and now we will start implementing
        

In [ ]:
# importing dependencies

from langchain.llms import OpenAI
#from langchain.chat_models import ChatOpenAI  #already imported
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
#from dotenv import load_dotenv   #already imported
import PyPDF2


In [ ]:
RESPONSE_JSON = {
        "1": {
             "mcq" : "multiple choice question",
            "options" : {
                    "a" : "choice here",
                    "b" : "choice here",
                    "c" : "choice here",
                    "d" : "choice here",
        `},
        "correct" : "correct answer",
    },
    "2": {
             "mcq" : "multiple choice question",
            "options" : {
                    "a" : "choice here",
                    "b" : "choice here",
                    "c" : "choice here",
                    "d" : "choice here",
        },
        "correct" : "correct answer",
    },
    "3": {
             "mcq" : "multiple choice question",
            "options" : {
                    "a" : "choice here",
                    "b" : "choice here",
                    "c" : "choice here",
                    "d" : "choice here",
        },
        "correct" : "correct answer",
    },
    "4": {
             "mcq" : "multiple choice question",
            "options" : {
                    "a" : "choice here",
                    "b" : "choice here",
                    "c" : "choice here",
                    "d" : "choice here",
        },
        "correct" : "correct answer",
    },
}

In [ ]:
TEMPLATE = """
TEXT: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple questions for {subject} student in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],       #here 'tone' is difficulty level
    template = TEMPLATE
)

In [ ]:
# now we will chain the llm and the prompt
quize_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)


# now we will write a prompt for evaluation

TEMPLATE2 = """
You are an expert English Grammarian and writer Professor. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity, if the quiz is not at per with the coginitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student 's ability.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [ ]:
# create a template for evaluation and this input will come from user side
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template = TEMPLATE)

In [ ]:
#create a chain for the above prompt
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key="review", verbose= True)

In [ ]:
# Now connecting both the chains using sequential chains
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text', "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose= True)

In [ ]:
file_path = r "C:\Users\imava\MCQgenerator\experiment\data.txt"

: 

In [ ]:
file_path

In [ ]:
# to read the data
with open(file_path, "r") as
    TEXT = file.read()

In [ ]:
# printing the content
print(TEXT)

In [ ]:
# NOTE the RESPONSE_Json is in the dictionary format; Serialise the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "Generative AI"
TONE = "medium"

In [ ]:
# generating final response

#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup Token usage Tracking in LangChain

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'text': TEXTt,
            "number" : NUMBER,
            "subject" : SUBJECT, 
            "tone" : TONE,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
#generated tokens cost
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost; {cb.total_cost}")

In [ ]:
#Generated Response
response

In [ ]:
# to get the questions for the quiz
quiz = response.get("quiz")

In [ ]:
# to get the questions in our desired json format which we have formatted
quiz = json.loads(quiz)

In [ ]:
# formatting the quiz questions segregating by "|"

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = "|".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ"; mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
# Converting it to the dataframe
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("GenerativeAI.csv", index=False)